Import Library

In [97]:
import os
import numpy as np
from PIL import Image
import numpy as np
import math
from numpy.random import normal

Flatten

In [98]:
# Set the input directory path
input_directory = "../Dataset/Foto_Resize_70x70"

# List all folders in the input directory
folders = [f for f in os.listdir(input_directory) if os.path.isdir(os.path.join(input_directory, f))]
print(f"Found folders: {folders}")

# Initialize arrays for inputs and outputs
X = []
Y = []

# Create a mapping from folder names to one-hot encoded labels
label_map = {folder: idx for idx, folder in enumerate(folders)}
num_classes = len(folders)

# Process each folder and photo
for folder in folders:
    folder_path = os.path.join(input_directory, folder)
    
    # Sort the list of photos to ensure consistent order
    photos = sorted([p for p in os.listdir(folder_path) if p.endswith(('.png', '.jpg', '.jpeg'))])
    
    for photo in photos:
        photo_path = os.path.join(folder_path, photo)
        image = Image.open(photo_path)
        
        # Convert the image to a numpy array and flatten it
        image_array = np.array(image).flatten()
        X.append(image_array)
        
        # Create a one-hot encoded label
        label = np.zeros(num_classes)
        label[label_map[folder]] = 1
        Y.append(label)

# Convert lists to numpy arrays
X = np.array(X)
Y = np.array(Y)

print(f"Input array shape: {X.shape}")
print(f"Output array shape: {Y.shape}")

Found folders: ['Azmira', 'David', 'Dimas', 'Fadhli', 'Fadlin', 'Hafidz', 'Haidar', 'Hanna', 'Keiko', 'Khansa', 'Mikhael', 'Puti', 'Raesa', 'Satwika', 'Toni']
Input array shape: (1384, 4900)
Output array shape: (1384, 15)


Normalize X, Print Y

In [99]:
print(X)
xmax = X.max()
xmin = X.min()
X =  (X-xmin)/(xmax-xmin)
print(X)

[[218 220 222 ... 137 112 103]
 [253 252 251 ... 137 127 116]
 [106 117 133 ... 220 219 218]
 ...
 [213 213 214 ...  55  50  48]
 [214 214 214 ...  70  68  68]
 [ 51  48  51 ... 213 213 212]]
[[0.85490196 0.8627451  0.87058824 ... 0.5372549  0.43921569 0.40392157]
 [0.99215686 0.98823529 0.98431373 ... 0.5372549  0.49803922 0.45490196]
 [0.41568627 0.45882353 0.52156863 ... 0.8627451  0.85882353 0.85490196]
 ...
 [0.83529412 0.83529412 0.83921569 ... 0.21568627 0.19607843 0.18823529]
 [0.83921569 0.83921569 0.83921569 ... 0.2745098  0.26666667 0.26666667]
 [0.2        0.18823529 0.2        ... 0.83529412 0.83529412 0.83137255]]


In [100]:
print(y)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


Initialize Layer

In [101]:
inputLayer = 2500
hiddenLayer = 1
outputLayer = 15

Calculating Weights for Input Layer to Hidden Layer

In [102]:
#70x70
#input = 4900, hiddenLayer = 1, neuron = 2

n = 4900
stddev = np.sqrt(2/n)
hiddenNeurons = 2
v = np.random.normal(loc=0, scale = stddev, size = (n, hiddenNeurons))
print(v)
np.shape(v)

[[ 0.00161475 -0.01605675]
 [-0.01382559 -0.0047221 ]
 [-0.06008978  0.02735197]
 ...
 [-0.00518433 -0.03201491]
 [-0.00077291 -0.02063725]
 [ 0.01725199  0.00036942]]


(4900, 2)

In [103]:
vb = np.random.normal(loc=0, scale = stddev, size = (1, hiddenNeurons))
print(vb)
np.shape(vb)

[[-0.01309486 -0.00012142]]


(1, 2)

Calculating Weights for Hidden Layer to Output Layer

In [104]:
#70x70
#input = 4900, hiddenLayer = 1, neuron = 2

n = 2
stddev = np.sqrt(2/n)
w = np.random.normal(loc=0, scale = stddev, size = (n, outputLayer))
print(w)
np.shape(w)

[[-0.40859299  0.39217189 -0.99085276  0.58094213 -0.68673176 -0.54812103
   0.78230152 -0.16417297  1.49664746 -0.34808117 -0.35699664  1.27525494
  -2.15028093 -0.85156063  0.8179641 ]
 [ 0.18611234 -0.3546493  -0.00370191 -0.13252137 -0.69719726 -0.76177509
   0.69542438 -1.73646764  0.75733751  0.63934573 -0.50149741  0.06427446
  -0.30879379  0.46015704  1.02586302]]


(2, 15)

Calculating Bias Weight from Hidden Layer to Output Layer

In [105]:
wb = np.random.normal(loc=0, scale = stddev, size = (1, outputLayer))
print(wb)
np.shape(wb)

[[ 0.01714073 -0.21289658  0.01032809 -0.39876341 -0.95590868  1.60369856
  -0.53549051  0.07062347  0.43956186 -0.06760634  0.47028145 -0.56084579
  -0.50375441  0.12364558  0.47702803]]


(1, 15)

Feedforward

In [106]:
z_in = np.matmul(X,v) + vb
print(z_in)
np.shape(z_in)

[[0.09031337 0.99964319]
 [0.14161072 0.56859291]
 [1.8233792  1.10239212]
 ...
 [0.87640572 1.3714234 ]
 [0.57352701 0.43214707]
 [0.94946663 1.01294877]]


(1384, 2)

In [ ]:
# z = max([0,z_in])
z = np.maximum(0,z_in)
print(z)

[[0.09031337 0.99964319]
 [0.14161072 0.56859291]
 [1.8233792  1.10239212]
 ...
 [0.87640572 1.3714234 ]
 [0.57352701 0.43214707]
 [0.94946663 1.01294877]]


In [114]:
#y_in = np.dot(z,w) + wb
y_in = np.matmul(z,w) + wb
print(y_in)

[[ 0.16628525 -0.53200098 -0.08285974 ... -1.00663713  0.50673112
   1.5763981 ]
 [ 0.06510173 -0.35901192 -0.13209216 ... -0.9838352   0.26469749
   1.17615895]
 [-0.52271046  0.11121889 -1.80045318 ... -4.76494378 -0.92179887
   3.09939007]
 ...
 [-0.08571369 -0.35556925 -0.86313782 ... -2.81175994  0.0084031
   2.600789  ]
 [-0.13677049 -0.14123607 -0.5595525  ... -1.87044293 -0.16589193
   1.38947623]
 [-0.18228242 -0.19978403 -0.93420338 ... -2.85816669 -0.21876732
   2.29280433]]


In [ ]:
# y = exponent/sumofexponent
exponents = [np.exp(i) for i in y_in]
sumofExponents = sum(exponents)
y = [(np.exp(i)/sumofExponents) for i in exponents]
print(y)
np.shape(y)

[array([0.00285776, 0.00146788, 0.00417331, 0.00126431, 0.00706033,
       0.00388941, 0.00110341, 0.00335564, 0.00222319, 0.00341602,
       0.00236806, 0.0006355 , 0.01133345, 0.00474893, 0.010247  ]), array([0.00255077, 0.0016401 , 0.00399278, 0.00133749, 0.00745717,
       0.00843128, 0.00082738, 0.00411615, 0.00096274, 0.00220791,
       0.00290793, 0.0006524 , 0.01142937, 0.00332377, 0.00207795]), array([1.58730596e-03, 2.49436171e-03, 1.96099520e-03, 3.62429245e-03,
       6.20570670e-03, 9.40907537e-04, 5.94978807e-02, 3.13171348e-03,
       3.12165952e+19, 1.67341094e-03, 1.49722077e-03, 1.69131103e-01,
       7.93144858e-03, 1.34428846e-03, 3.50159982e+05]), array([1.60012727e-03, 2.84293280e-03, 2.10269307e-03, 3.03597684e-03,
       6.55655629e-03, 2.19451936e-03, 3.60440021e-03, 4.24223360e-03,
       8.81118334e+03, 1.26597320e-03, 2.00981593e-03, 1.46340509e-02,
       8.04397667e-03, 1.33235559e-03, 2.64242403e-01]), array([0.00271483, 0.00146749, 0.00348192, 0.00134529

(1384, 15)

Back Propagation

In [ ]:
#setiap unit output
#sigma_k = (Y-y)*f'(y_in)
#gradien fungsi softmax = 